### Import

In [ ]:
import sys
sys.path.append('/home/lyu/AI/AImers/6th')
import pandas as pd
import numpy as np
from features.features import categorical_columns, numeric_columns
from sklearn.preprocessing import  OrdinalEncoder


### Data Load

In [ ]:
DATA_PATH = '/home/lyu/AI/AImers/6th/data'
train = pd.read_csv(DATA_PATH + '/train.csv').drop(columns=['ID'])
test = pd.read_csv(DATA_PATH + '/test.csv').drop(columns=['ID'])

### 횟수 columns

In [ ]:
# Create a function to convert the string to numeric value
def convert_to_numeric(x):
    if pd.isna(x):
        return x
    if '회 이상' in str(x):
        return 6  # '6회 이상'은 6으로 처리
    return int(str(x).replace('회', ''))

# Print column names that end with '횟수'
count_cols = [col for col in train.columns if col.endswith('횟수')]
for col in count_cols:
    train[col] = train[col].apply(convert_to_numeric)

for col in count_cols:
    test[col] = test[col].apply(convert_to_numeric)

train['총 시술 횟수'] = train['DI 시술 횟수'] + train['IVF 시술 횟수']
test['총 시술 횟수'] = test['DI 시술 횟수'] + test['IVF 시술 횟수']




## 데이터 분할

In [ ]:
train_ivf = train[train['시술 유형'] == 'IVF']
train_di = train[train['시술 유형'] == 'DI']

test_ivf = test[test['시술 유형'] == 'IVF']
test_di = test[test['시술 유형'] == 'DI']

분할 후 의미없는 컬럼 제거

In [ ]:
# Drop columns with 100% null values in train_ivf and train_di
null_cols_ivf = [col for col in train_ivf.columns if train_ivf[col].isnull().all()]
null_cols_di = [col for col in train_di.columns if train_di[col].isnull().all()]

train_ivf = train_ivf.drop(columns=null_cols_ivf)
train_di = train_di.drop(columns=null_cols_di)

test_ivf = test_ivf.drop(columns=null_cols_ivf)
test_di = test_di.drop(columns=null_cols_di)

print("Dropped columns in train_ivf:", null_cols_ivf)
print("Dropped columns in train_di:", null_cols_di)

for name, df, test_df in [("train_ivf", train_ivf, test_ivf), ("train_di", train_di, test_di)]:
    unique_cols = [col for col in df.columns if df[col].nunique(dropna=False) == 1]
    df.drop(columns=unique_cols, inplace=True)
    test_df.drop(columns=unique_cols, inplace=True)
    print(f"Dropped columns in {name}:", unique_cols)

Dropped columns in train_ivf: []
Dropped columns in train_di: ['단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '배아 생성 주요 이유', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']
Dropped columns in train_ivf: ['시술 유형', '불임 원인 - 여성 요인']
Dropped columns in train_di: ['시술 유형', '배란 유도 유형', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '난자 출처', '정자 출처', '난자 기증자 나이']


결측치 및 수치형 카테고리화

In [ ]:
ivf_null_cols = [col for col in train_ivf.columns if train_ivf[col].isnull().any()]
di_null_cols = [col for col in train_di.columns if train_di[col].isnull().any()]

ivf_cat_cols = list(set(train_ivf.columns.to_list()) & (set(categorical_columns) | set(ivf_null_cols)) - set(count_cols))
ivf_num_cols = list(set(train_ivf.columns.to_list()) & (set(numeric_columns) - set(ivf_null_cols)) | set(count_cols))
di_cat_cols = list(set(train_di.columns.to_list()) & (set(categorical_columns) | set(di_null_cols))  - set(count_cols))
di_num_cols = list(set(train_di.columns.to_list()) & (set(numeric_columns) - set(di_null_cols))  | set(count_cols))

train_ivf["임신 시도 또는 마지막 임신 경과 연수"] = train_ivf["임신 시도 또는 마지막 임신 경과 연수"].astype(object)
train_ivf.loc[train_ivf["임신 시도 또는 마지막 임신 경과 연수"].between(0, 20, inclusive="both"), "임신 시도 또는 마지막 임신 경과 연수"] = "0~20"
train_ivf["난자 혼합 경과일"] = train_ivf["난자 혼합 경과일"].astype(object)
train_ivf.loc[train_ivf["난자 혼합 경과일"].between(1, 7, inclusive="both"), "난자 혼합 경과일"] = "1~7"
train_ivf["배아 해동 경과일"] = train_ivf["배아 해동 경과일"].astype(object)
train_ivf.loc[train_ivf["배아 해동 경과일"].between(1, 7, inclusive="both"), "배아 해동 경과일"] = "1~7"


test_ivf["임신 시도 또는 마지막 임신 경과 연수"] = test_ivf["임신 시도 또는 마지막 임신 경과 연수"].astype(object)
test_ivf.loc[test_ivf["임신 시도 또는 마지막 임신 경과 연수"].between(0, 20, inclusive="both"), "임신 시도 또는 마지막 임신 경과 연수"] = "0~20"
test_ivf["난자 혼합 경과일"] = test_ivf["난자 혼합 경과일"].astype(object)
test_ivf.loc[test_ivf["난자 혼합 경과일"].between(1, 7, inclusive="both"), "난자 혼합 경과일"] = "1~7"
test_ivf["배아 해동 경과일"] = test_ivf["배아 해동 경과일"].astype(object)
test_ivf.loc[test_ivf["배아 해동 경과일"].between(1, 7, inclusive="both"), "배아 해동 경과일"] = "1~7"

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in ivf_cat_cols:
    if col in train_ivf.columns:
        train_ivf[col] = train_ivf[col].astype(str)
        test_ivf[col] = test_ivf[col].astype(str)

for col in di_cat_cols:
    if col in train_di.columns:
        train_di[col] = train_di[col].astype(str)
        test_di[col] = test_di[col].astype(str)


## 훈련

In [ ]:
from supervised.automl import AutoML
automl = AutoML(mode="Compete",
                algorithms = ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Nearest Neighbors'],
                n_jobs = -1,total_time_limit=50000, eval_metric="auc", ml_task = "binary_classification",)


/home/lyu/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


In [ ]:
X = train_di.drop(columns=['임신 성공 여부'])
y = train_di['임신 성공 여부']

automl.fit(X, y)

AutoML directory: AutoML_3
The task is binary_classification with evaluation metric auc
AutoML will use algorithms: ['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM', 'Xgboost', 'CatBoost', 'Nearest Neighbors']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'mix_encoding', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree auc 0.619285 trained in 2.77 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle,Stratify
* Step simple_algorithms will try to check up to 3 models
1_DecisionTree auc 0.668048 trained in 5.78 seconds
2_DecisionTree auc 0.673056 trained in 5.7 seconds
3_DecisionTree auc 0.673051 trained in 5.78 seconds
* Step 

/home/lyu/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/lyu/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/lyu/.local/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/home/lyu/.local/lib/python3.10/site-pac

None 10
Add Golden Feature: DI 출산 횟수_ratio_총 임신 횟수
Add Golden Feature: 총 임신 횟수_ratio_DI 출산 횟수
Add Golden Feature: DI 임신 횟수_ratio_DI 출산 횟수
Add Golden Feature: DI 출산 횟수_ratio_DI 임신 횟수
Add Golden Feature: 총 임신 횟수_ratio_총 출산 횟수
Add Golden Feature: 총 출산 횟수_ratio_총 임신 횟수
Add Golden Feature: DI 시술 횟수_ratio_총 출산 횟수
Add Golden Feature: 총 출산 횟수_sum_DI 임신 횟수
Add Golden Feature: DI 출산 횟수_multiply_DI 임신 횟수
Add Golden Feature: 총 출산 횟수_ratio_DI 시술 횟수
Created 10 Golden Features in 8.61 seconds.
54_ExtraTrees_GoldenFeatures auc 0.690511 trained in 41.23 seconds
50_ExtraTrees_GoldenFeatures auc 0.688212 trained in 26.34 seconds
53_ExtraTrees_GoldenFeatures auc 0.69077 trained in 28.2 seconds
* Step kmeans_features will try to check up to 3 models
54_ExtraTrees_KMeansFeatures auc 0.6898 trained in 37.26 seconds
50_ExtraTrees_KMeansFeatures auc 0.688109 trained in 33.27 seconds
53_ExtraTrees_KMeansFeatures auc 0.688452 trained in 32.11 seconds
* Step insert_random_feature will try to check up to 1 model
5

/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 49884 (\N{HANGUL SYLLABLE SI}) missing from current font.
/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 49696 (\N{HANGUL SYLLABLE SUL}) missing from current font.
/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 45817 (\N{HANGUL SYLLABLE DANG}) missing from current font.
/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 45208 (\N{HANGUL SYLLABLE NA}) missing from current font.
/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 51060 (\N{HANGUL SYLLABLE I}) missing from current font.
/home/lyu/.local/lib/python3.10/site-packages/supervised/utils/additional_metrics.py:701: UserWarning: Glyph 52636 (\N{HANGUL SYLLABLE CUL}) missing from current f

Drop features ['임신 시도 또는 마지막 임신 경과 연수', '불임 원인 - 배란 장애', '불임 원인 - 난관 질환', '총 시술 횟수', '부부 부 불임 원인', '시술 시기 코드', '불명확 불임 원인', 'random_feature', '여성 주 불임 원인', '불임 원인 - 정자 형태', '여성 부 불임 원인', '불임 원인 - 자궁내막증', 'IVF 출산 횟수', '불임 원인 - 정자 농도', '클리닉 내 총 시술 횟수', '부부 주 불임 원인', '특정 시술 유형']
* Step features_selection will try to check up to 5 models
54_ExtraTrees_SelectedFeatures auc 0.695659 trained in 24.01 seconds
30_CatBoost_SelectedFeatures auc 0.687556 trained in 16.94 seconds
7_Default_RandomForest_SelectedFeatures auc 0.691193 trained in 25.97 seconds
12_Xgboost_SelectedFeatures auc 0.692741 trained in 13.93 seconds
20_LightGBM_SelectedFeatures auc 0.682563 trained in 13.89 seconds
* Step hill_climbing_1 will try to check up to 27 models
62_ExtraTrees_SelectedFeatures auc 0.693305 trained in 30.17 seconds
63_ExtraTrees_SelectedFeatures auc 0.699521 trained in 22.64 seconds
64_ExtraTrees auc 0.68937 trained in 28.34 seconds
65_ExtraTrees auc 0.689602 trained in 28.76 seconds
66_Xgboost_Selected

AutoML(algorithms=['Decision Tree', 'Random Forest', 'Extra Trees', 'LightGBM',
                   'Xgboost', 'CatBoost', 'Nearest Neighbors'],
       eval_metric='auc', ml_task='binary_classification', mode='Compete',
       total_time_limit=50000)

In [ ]:
pred_ivf = 0
pred_di = automl.predict_proba(test_di)[:, 1]

In [ ]:
model_ivf = AutoML(results_path="AutoML_2")
pred_ivf = model_ivf.predict_proba(test_ivf)[:, 1]

In [ ]:
print(train_ivf["임신 성공 여부"].mean(), train_di["임신 성공 여부"].mean())
print(pred_ivf.mean(), pred_di.mean())

0.2616052147484604 0.12891432204736925
0.26165149605805804 0.1383915785866596


### Submission

In [ ]:
test_ivf = test[test['시술 유형'] == 'IVF']
test_di = test[test['시술 유형'] == 'DI']

In [ ]:
sample_submission = pd.read_csv(DATA_PATH + '/sample_submission.csv')
sample_submission.loc[test_ivf.index, 'probability'] = pred_ivf
sample_submission.loc[test_di.index, 'probability'] = pred_di

In [ ]:
sample_submission.to_csv('../automl.csv', index=False)

In [ ]:
sample_submission['probability'].max(), sample_submission['probability'].min()

(0.7382358203023288, 0.0005671814892622001)

In [ ]:
pred_di.max(), pred_di.min(), pred_ivf.max(), pred_ivf.min()

(0.39218501829312197,
 0.03977252208582412,
 0.7382358203023288,
 0.0005671814892622001)